from Kaggle:
https://www.kaggle.com/c/cat-in-the-dat/overview

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression

In [3]:
data_train = pd.read_csv('./data/train.csv',index_col='id')
X = data_train.drop(columns='target')
y = data_train.loc[:,'target']

X.shape

(300000, 23)

## Usando o Label Encoder

In [4]:
label = LabelEncoder()
for c in  X.columns:
    if(X[c].dtype=='object'):
        X[c]=label.fit_transform(X[c])
    else:
        X[c]=X[c]

In [ ]:
data_train.columns

In [ ]:
X.columns

## Usando onehotencoder


In [ ]:
ohe = OneHotEncoder()
ohe.fit(X)
X_encoded = ohe.transform(X)
X_encoded.shape

## Regressão Logística

In [5]:
lr = LogisticRegression()
lr.fit(X,y)

C:\Users\jonatas.hernandes\.conda\envs\s2\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [6]:
data_test = pd.read_csv('./data/test.csv',index_col='id')

X_test = data_test

for c in  X_test.columns:
    if(X_test[c].dtype=='object'):
        X_test[c]=label.fit_transform(X_test[c])
    else:
        X_test[c]=X_test[c]
        

In [8]:
y_result = lr.predict(X_test)

In [10]:
pd.DataFrame(data=y_result,index=X_test.index,columns=['target']).to_csv('output.csv')